요약 기반 명사, 토픽 기반 명사 데이터를 활용해 하나의 도서를 좋아요 누르면 그것과 나머지 도서들과의 코사인 유사도를 분석하고 상위 10개 추천. 그리고 그 추천 데이터 txt로 저장
+ 좋아요 한 도서가 여러개 일 때 별점 정도에 따라 반영 비중을 다르게 하기.
+ 실험 후 1400개의 전체 데이터로 늘려 진행

https://wikidocs.net/24603

#데이터 준비

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import os
summary_path = "/content/drive/MyDrive/교보데이터 박병석 원본/명사추출 데이터/추천단어_중복제거(요약기반).csv"
lda_path = "/content/drive/MyDrive/교보데이터 박병석 원본/명사추출 데이터/추천단어_중복제거(LDA기반).csv"

summary_df = pd.read_csv(summary_path, encoding='cp949')
lda_df = pd.read_csv(lda_path, encoding='cp949')

summary_df = summary_df.drop('추천단어',axis=1)
original_df = pd.merge(summary_df, lda_df[['책 이름', '추천단어']], on='책 이름', how='left')
original_df.rename(columns={'nouns': '요약 기반 명사','추천단어': 'LDA 기반 명사'}, inplace=True) #summary 기반 - nouns, lda 기반 - 추천 단어
original_df.rename(columns={}, inplace=True)
original_df.head(3)

,책 이름,작가,장르,ISBN,책 소개,요약 기반 명사,LDA 기반 명사
0,모순,양귀자,국내도서\n > \n소설\n > \n한국소설\n > \n한국소설일반,9.790000e+12,※본 도서의 표지 색상은 2쇄를 주기로 변경됩니다. 등록된 이미지와 다른 색의 표지...,"소설,인생,모순,양귀자,탐구,사람,이야기,베스트셀러,계기,일상,미혼여성,마련,에피소...","처리,소설,사람,보스,양장본,양귀자,초판,어머니,투성이,색상,미혼여성,계기,문장,이..."
1,죽이고 싶은 아이 2,이꽃님,국내도서\n > \n청소년\n > \n청소년 소설\n > \n한국,9.790000e+12,“이 이야기를 쓰지 않고서는 다른 작품을 쓰기 어렵겠다고 생각했다.”한국 청소년 문...,"압도,단숨,산산조각,꽃님,전개,희망,믿음,몰입,절망,이야기,독자,진실,청소년","한국,전개,독자,눈물,퍼즐,순간,쓰기,감동,꽃님,한순간,이유,진실,문학,믿음,결말,..."
2,우리에게 남은 시간 46일,이설,국내도서\n > \n소설\n > \n한국소설\n > \n한국소설일반,9.790000e+12,"〈우리에게 남은 시간이 46일〉 이 소설은 시한부인 해인의 곁에서 슬퍼하며, 끝까지...","우현,인상,사랑,작품,이야기","우현,지지,절대,부인,소설,이야기,독자,작품,마지막,시한,해인,사랑,인상,포기,마음"


실제 필요한 열만 남기기

In [3]:
df = original_df.drop(['작가','장르','ISBN','책 소개'],axis=1)
df.head(3)

,책 이름,요약 기반 명사,LDA 기반 명사
0,모순,"소설,인생,모순,양귀자,탐구,사람,이야기,베스트셀러,계기,일상,미혼여성,마련,에피소...","처리,소설,사람,보스,양장본,양귀자,초판,어머니,투성이,색상,미혼여성,계기,문장,이..."
1,죽이고 싶은 아이 2,"압도,단숨,산산조각,꽃님,전개,희망,믿음,몰입,절망,이야기,독자,진실,청소년","한국,전개,독자,눈물,퍼즐,순간,쓰기,감동,꽃님,한순간,이유,진실,문학,믿음,결말,..."
2,우리에게 남은 시간 46일,"우현,인상,사랑,작품,이야기","우현,지지,절대,부인,소설,이야기,독자,작품,마지막,시한,해인,사랑,인상,포기,마음"


#요약 기반 명사 기준 코사인유사도 검출


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer()

In [6]:
#요약 기반 - 벡터화, 코사인 유사도 매트릭스
sum_list = df['요약 기반 명사'].apply(lambda x: ' '.join(x.split(',')))
sum_tfidf_matrix = vectorizer.fit_transform(sum_list)
sum_cosine_sim = cosine_similarity(sum_tfidf_matrix,sum_tfidf_matrix) #cosine similarity

#LDA 기반 - 벡터화, 코사인 유사도 매트릭스
lda_list = df['LDA 기반 명사'].apply(lambda x: ' '.join(x.split(',')))
lda_tfidf_matrix = vectorizer.fit_transform(lda_list)
lda_cosine_sim = cosine_similarity(lda_tfidf_matrix,lda_tfidf_matrix) #cosine similarity

#책 이름:인덱스 딕셔너리 생성
sum_title_to_index = dict(zip(df['책 이름'], df.index))
lda_title_to_index = dict(zip(df['책 이름'], df.index))

In [7]:
def get_recommendations(title, cosine_sim, title_to_index):
    # 선택한 영화의 타이틀로부터 해당 영화의 인덱스를 받아온다.
    idx = title_to_index[title]

    # 해당 영화와 모든 영화와의 유사도를 가져온 후 유사도에 따라 영화들을 정렬한다.
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아온다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 얻는다.
    movie_indices = [idx[0] for idx in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴한다.
    return df['책 이름'].iloc[movie_indices]

제목/ISBN 입력시 추천해주는 서비스 완성
(지금은 있는 데이터로만 할 수 밖에 없음. 책 제목/ISBN을 바로 검색해서 책 소개를 가져와 명사추출을 한 다음 그거랑 코사인 유사도를 분석하지 않는 이상.)

In [12]:
num = 141
print('<<', df['책 이름'].loc[num], '>> 관련 도서 추천')
get_recommendations(df['책 이름'].loc[num], lda_cosine_sim, lda_title_to_index)

<< 초판본 싯다르타(리커버 한정판) >> 관련 도서 추천


,책 이름
138,데미안(리커버 한정판)(블랙벨벳 에디션)
205,나의 라임오렌지나무(40주년 기념 스페셜 에디션)
135,데미안(리커버 한정판)
131,변신(1916년 오리지널 초판본 표지디자인)
174,그리스인 조르바
181,그리스인 조르바
126,데미안: 오리지널 초판본 표지디자인
134,젊은 베르테르의 슬픔
123,반항하는 인간
128,수레바퀴 아래서: 오리지널 초판본 표지디자인


In [11]:
get_recommendations(df['책 이름'].loc[num], sum_cosine_sim, sum_title_to_index)

,책 이름
138,데미안(리커버 한정판)(블랙벨벳 에디션)
135,데미안(리커버 한정판)
126,데미안: 오리지널 초판본 표지디자인
124,데미안
131,변신(1916년 오리지널 초판본 표지디자인)
123,반항하는 인간
133,수레바퀴 아래서
173,무의미의 축제
84,이반 일리치의 죽음
85,이반 일리치의 죽음
